# TODOs

* PCA to see which feature has how much influence
* implement scoring function, eating food gives plus, getting smaller distance to food gives plus, reducing distance to enemy wolf gives minus etc.?

In [95]:
import pandas as pd
from sklearn import tree, naive_bayes, svm
import ast
import math
import pickle
import os
import glob

## Use the same config as in the later game

In [96]:
#[General_Constants]
FIELD_WIDTH = 19
FIELD_HEIGHT = 15

#[Game_Constants]
NO_ITERATIONS = 100
MAX_CALC_TIME = 100000

#[Field_Constants]
CELL_EMPTY = '.'
CELL_SHEEP_1 = 'S'
CELL_SHEEP_1_d = 'U'
CELL_WOLF_1 = 'W'
CELL_SHEEP_2 = 's'
CELL_SHEEP_2_d = 'u'
CELL_WOLF_2 = 'w'
CELL_GRASS = 'g'
CELL_RHUBARB = 'r'
CELL_FENCE = '#'


#[Movements]
MOVE_NONE = 0
MOVE_UP = -1
MOVE_DOWN = 1
MOVE_LEFT = -2
MOVE_RIGHT = 2

#[Awards]
AWARD_RHUBARB = 5
AWARD_GRASS = 1

# Load data

Load all games into one dataframe

In [97]:
# path = "../example_player/training_data" 
# all_files = glob.glob(path + "/*.csv")

# li = []

# for filename in all_files:
#     df = pd.read_csv(filename, index_col=None, header=0, keep_default_na=False)    
#     df['game'] = filename
#     li.append(df)

# frame = pd.concat(li, axis=0, ignore_index=True)

In [140]:
path = "../training_data/subset1" 
all_files = glob.glob(path + "/*.csv")   

training_data = []

#load the data into a pandas frames
for file in all_files:
    game_data = pd.read_csv(file,index_col=False)
    reason = game_data.iloc[-1][6]
    
    #if the reason is found, add it to each line to fill out the blanks
    if type(reason) is str:
        for index,row in game_data.iterrows():
            game_data.loc[index,'reason'] = reason

    #else there was no reason, implying the game reached the number of iterations
    else:
        for index,row in game_data.iterrows():
            game_data.loc[index,'reason'] = 'max_iterations'    
    
    training_data.append(game_data)

#preview the final 5 lines
training_data[-1].head()

,field_before,field_after,turn_made_by,move_made,score1,score2,reason
0,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 sheep,1,0,0,max_iterations
1,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player2 sheep,-2,0,0,max_iterations
2,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 sheep,1,0,0,max_iterations
3,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player2 sheep,-1,0,0,max_iterations
4,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 wolf,2,0,0,max_iterations


# Feature selection and Instance selection 

Import the Python file, such that I can reuse all the preprocessing function and avoid duplicate code

## Sheep

I used a value function for each possible move (up/down/left/right/stay). Value is dependent on how good the position would be (food/distance to wolf etc.). Based on this value function that contains some "spirit" of assignment 1, I train which move should be done to learn from the behavior of the Search Algorithms used in assignment 1.

sheep_coordinates[column,row]

In [141]:
training_data[0]

,field_before,field_after,turn_made_by,move_made,score1,score2,reason
0,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 sheep,1,0,0,sheep2 eaten
1,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player2 sheep,-1,0,0,sheep2 eaten
2,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 sheep,-2,0,0,sheep2 eaten
3,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player2 sheep,2,0,0,sheep2 eaten
4,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 wolf,2,0,0,sheep2 eaten
5,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player2 wolf,-2,0,0,sheep2 eaten
6,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 sheep,-2,0,0,sheep2 eaten
7,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player2 sheep,2,0,0,sheep2 eaten
8,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 sheep,-2,1,0,sheep2 eaten
9,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player2 sheep,2,1,1,sheep2 eaten


In [174]:
from importlib import reload  # to get changes in code
import chriweb_a2
chriweb_a2 = reload(chriweb_a2)
ii = chriweb_a2.IntrepidIbex()

X_sheep = []
Y_sheep = []
number_moves = 0

for game in training_data:
    
    #we want to learn from the winning player, which is the player with the highest score:
    if game.iloc[-1][4] < game.iloc[-1][5]:
        sheep_label = 's'
        wolf_label = 'W'
        figure = 1
    
    elif game.iloc[-1][4] > game.iloc[-1][5]:
        sheep_label = 'S'
        wolf_label = 'w'
        figure = 2
    else:
        continue
        
    #for each game state in our training data
    for index,row in game.iterrows():

        #we don't want games that ended because of an error or because the sheep commited suicide
        if row['reason'] not in ('sheep1 eaten','sheep2 eaten','max_iterations'):
            continue

        #we want to only learn from sheep
        if row['turn_made_by'] not in ('player1 sheep','player2 sheep'):
            continue
            
        #we want to only learn from winning player
        if str(figure) not in row['turn_made_by']:
            continue
        
        number_moves += 1
        
        #this is the move that we are learning from this game state
        move = row['move_made']

        #create empty feature array for this game state
        game_features = []

        #turn the field from before the move from a string back to a list
        field = ast.literal_eval(row['field_before'])     
                
        game_features = ii.get_features_sheep(figure, field)  
        
        #add features and move to X_sheep and Y_sheep
        X_sheep.append(game_features)
        Y_sheep.append(move)  

In [175]:
#this prints an example of our feature and outcome vector:
print(X_sheep[:10])
print(Y_sheep[:10])
print(number_moves)

[[10.330412131161864, 13.174889915651544, 50, 50, 10.330412131161864], [9.469648806522665, 15.848931924611133, 50, 12.125732532083184, 9.469648806522665], [12.507807439045607, 13.966610165238235, 50, 13.966610165238235, 5.571874686029346], [15.848931924611133, 15.848931924611133, 50, 1000, 2.3926754832146493], [13.966610165238235, 5.613892585058365, 50, 1000, 1000], [16.505554636533898, 2.4435886761032473, 50, 15.848931924611133, 5.533519108533716], [17.769336928223957, 1000, 50, 7.256214310086261, 1000], [19.725021954206706, 19.725021954206706, 50, 15.848931924611133, 5.551598955570587], [17.769336928223957, 1000, 50, 1000, 3.930516026927031], [19.759342353974255, 2.4204440678156667, 50, 2.4204440678156667, 4.4751649892234715]]
[-1, 2, 2, 2, -1, -1, 1, 2, 2, 1]
1530


In [176]:
Y_sheep_f = (pd.DataFrame(X_sheep).idxmin(axis=1) - 2).values

In [177]:
df_investigate_s = pd.DataFrame(X_sheep)
df_investigate_s['y'] = Y_sheep
df_investigate_s['y_f'] = Y_sheep_f
df_investigate_s

,0,1,2,3,4,y,y_f
0,10.330412,13.174890,50.000000,50.000000,10.330412,-1,-2
1,9.469649,15.848932,50.000000,12.125733,9.469649,2,-2
2,12.507807,13.966610,50.000000,13.966610,5.571875,2,2
3,15.848932,15.848932,50.000000,1000.000000,2.392675,2,2
4,13.966610,5.613893,50.000000,1000.000000,1000.000000,-1,-1
5,16.505555,2.443589,50.000000,15.848932,5.533519,-1,-1
6,17.769337,1000.000000,50.000000,7.256214,1000.000000,1,1
7,19.725022,19.725022,50.000000,15.848932,5.551599,2,2
8,17.769337,1000.000000,50.000000,1000.000000,3.930516,2,2
9,19.759342,2.420444,50.000000,2.420444,4.475165,1,-1


## Wolf

Explain here in text which feature you used for the wolf, and which data you used for training.

In [164]:
from importlib import reload  # to get changes in code
import chriweb_a2
chriweb_a2 = reload(chriweb_a2)
ii = chriweb_a2.IntrepidIbex()

X_wolf = []
Y_wolf = []
number_moves = 0

for game in training_data:
    
    #we want to learn from the winning player, which is the player with the highest score:
    if game.iloc[-1][4] < game.iloc[-1][5]:
        sheep_label = 's'
        wolf_label = 'W'
        figure = 1
    
    elif game.iloc[-1][4] > game.iloc[-1][5]:
        sheep_label = 'S'
        wolf_label = 'w'
        figure = 2
    else:
        continue

    rhubarb = 'r'
    grass = 'g'

    #for each game state in our training data
    for index,row in game.iterrows():

        #we don't want games that ended because of an error or because the sheep commited suicide
        if row['reason'] not in ('sheep1 eaten','sheep2 eaten','max_iterations'):
            continue

        #we want to only learn from sheep
        if row['turn_made_by'] not in ('player1 wolf','player2 wolf'):
            continue
            
        #we want to only learn from winning player
        if str(figure) not in row['turn_made_by']:
            continue
        
        number_moves += 1
        
        #this is the move that we are learning from this game state
        move = row['move_made']

        #create empty feature array for this game state
        game_features = []

        #turn the field from before the move from a string back to a list
        field = ast.literal_eval(row['field_before'])     
        
        game_features = ii.get_features_wolf(figure, field)  
        
        #add features and move to X_sheep and Y_sheep
        X_wolf.append(game_features)
        Y_wolf.append(move)  

In [165]:
#this prints an example of our feature and outcome vector:
print(X_wolf[:10])
print(Y_wolf[:10])
print(number_moves)

[[1, 1000, 50, 50, 1000], [50, 1, 50, 50, 50], [1, 50, 50, 50, 50], [1, 50, 50, 50, 50], [1, 50, 50, 50, 50], [1, 50, 50, 50, 50], [1, 50, 50, 50, 50], [1, 50, 50, 50, 50], [1, 50, 50, 50, 50], [1000, 1000, 50, 1, 50]]
[-2, -2, -1, -1, -2, -1, -2, -2, -1, 2]
764


In [166]:
Y_wolf_f = (pd.DataFrame(X_wolf).idxmin(axis=1) - 2).values

In [167]:
df_investigate_w = pd.DataFrame(X_wolf)
df_investigate_w['y'] = Y_wolf
df_investigate_w['y_f'] = Y_wolf_f
df_investigate_w

,0,1,2,3,4,y,y_f
0,1,1000,50,50,1000,-2,-2
1,50,1,50,50,50,-2,-1
2,1,50,50,50,50,-1,-2
3,1,50,50,50,50,-1,-2
4,1,50,50,50,50,-2,-2
5,1,50,50,50,50,-1,-2
6,1,50,50,50,50,-2,-2
7,1,50,50,50,50,-2,-2
8,1,50,50,50,50,-1,-2
9,1000,1000,50,1,50,2,1


# Build Sheep Model

Min-Max-normalizing for all values

In [117]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# X_sheep_norm=scaler.fit_transform(X_sheep)

PCA (https://stackoverflow.com/questions/50796024/feature-variable-importance-after-a-pca-analysis)

In [118]:
# from sklearn import decomposition
# pca = decomposition.PCA(n_components=3)
# pca.fit(X_sheep_norm)
# print(pca.explained_variance_ratio_)
# print(abs( pca.components_ ))


Split into train and test data

In [105]:
# X_sheep

In [178]:
from sklearn import model_selection
x_train_s, x_test_s, y_train_s, y_test_s = model_selection.train_test_split(X_sheep, Y_sheep_f, test_size=0.25)

## Train & Test sheep

Allowed algorithms:
* Naive Bayes (GaussianNB, MultinominalNB, ComplementNB, BernoulliNB)
* Decision Tree
* Support Vector Machine (SVC, NuSVC, LinearSVC)

In [179]:
from sklearn.metrics import confusion_matrix
import numpy as np

## Naive Bayes

In [108]:
# sheep_NB = naive_bayes.GaussianNB()
# # sheep_NB = naive_bayes.MultinomialNB()
# # sheep_NB = naive_bayes.ComplementNB()
# # sheep_NB = naive_bayes.BernoulliNB()
# sheep_NB = sheep_NB.fit(x_train_s,y_train_s)
# y_pred_s = sheep_NB.predict(x_test_s)

# cm = confusion_matrix(y_test_s, y_pred_s.tolist())
# recall = np.diag(cm) / np.sum(cm, axis = 1)
# precision = np.diag(cm) / np.sum(cm, axis = 0)

# print(cm)
# print(np.mean(recall))
# print(np.mean(precision))

## Decision Tree

In [180]:
sheep_tree = tree.DecisionTreeClassifier()
sheep_tree = sheep_tree.fit(x_train_s,y_train_s)
y_pred_s = sheep_tree.predict(x_test_s)

cm = confusion_matrix(y_test_s, y_pred_s.tolist())
recall = np.diag(cm) / np.sum(cm, axis = 1)
precision = np.diag(cm) / np.sum(cm, axis = 0)

print(cm)
print(np.mean(recall))
print(np.mean(precision))

[[150   2   0   4   3]
 [  3  82   0   2   2]
 [  0   0   3   0   0]
 [  5   5   0  62   3]
 [  3   1   0   2  51]]
0.9172296099587532
0.9185818390239909


## SVM

In [131]:
# sheep_svm = svm.SVC()
# # sheep_svm = svm.NuSVC()
# # sheep_svm = svm.LinearSVC()
# sheep_svm = sheep_svm.fit(x_train_s,y_train_s)
# y_pred_s = sheep_svm.predict(x_test_s)

# cm = confusion_matrix(y_test_s, y_pred_s.tolist())
# recall = np.diag(cm) / np.sum(cm, axis = 1)
# precision = np.diag(cm) / np.sum(cm, axis = 0)

# print(cm)
# print(np.mean(recall))
# print(np.mean(precision))

[[10  0  0  0]
 [ 5  0  0  0]
 [ 6  0  0  0]
 [ 3  0  0  1]]
0.3125
nan


/home/chris/PAI/Ex2/venv/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/chris/PAI/Ex2/venv/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


# Train wolf

## Decision Tree

In [171]:
from sklearn import model_selection
x_train_w, x_test_w, y_train_w, y_test_w = model_selection.train_test_split(X_wolf, Y_wolf_f, test_size=0.25)

In [172]:
wolf_tree = tree.DecisionTreeClassifier()
wolf_tree = wolf_tree.fit(x_train_w,y_train_w)
y_pred_w = wolf_tree.predict(x_test_w)

cm = confusion_matrix(y_test_w, y_pred_w.tolist())
recall = np.diag(cm) / np.sum(cm, axis = 1)
precision = np.diag(cm) / np.sum(cm, axis = 0)

print(cm)
print(np.mean(recall))
print(np.mean(precision))

[[51  0  0  0]
 [ 0 45  0  0]
 [ 0  0 48  0]
 [ 0  0  0 47]]
1.0
1.0


# Save models to files

Save your models to files here using pickle. Change the [uzhshortname] to your own UZH shortname. This name needs to match the model that you caller in your python player file.

In [181]:
sheep_filename = 'chriweb_sheep_model.sav'
wolf_filename = 'chriweb_wolf_model.sav'

pickle.dump(sheep_tree,open(sheep_filename,'wb'))
pickle.dump(wolf_tree,open(wolf_filename,'wb'))

# MISC

## Test prediction for fixed values

In [159]:
sheep_tree.predict([[1.6183970063527977, 50, 50, 1.0, 50]])

array([-2])

In [ ]:
<class 'list'>: [1.6183970063527977, 50, 50, 1.0, 50]